<a href="https://colab.research.google.com/github/KK-marius/Project-NLP-bag-of-words/blob/main/Text_classification_with_RNN_and_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification with Neural Networks

### Question
> * NAME : KOUAKOU
> * SURNAME : Marius

## Imports & Usefull Tools

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
import shutil

import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import matplotlib.pyplot as plt

In [ ]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([f'test {metric}', f'validation {metric}'])

def fit_and_test(model, train_dataset, validation_dataset, epochs):
  history = model.fit(train_dataset, epochs=epochs,
                      validation_data=validation_dataset,
                      #validation_steps=10
                      )

  validation_loss, validation_acc = model.evaluate(validation_dataset)

  print('Validation Loss:', validation_loss)
  print('Validation Accuracy:', validation_acc)

  # sample_text = ('The movie was cool. The animation and the graphics '
  #               'were out of this world. I would recommend this movie.')
  # predictions = model.predict(np.array([sample_text]))
  # for text, pred in zip([sample_text], predictions):
  #   print(f"{text:10} >> {pred}")

  plt.figure(figsize=(16, 8))
  plt.subplot(1, 2, 1)
  plot_graphs(history, 'accuracy')
  plt.ylim(None, 1)
  plt.subplot(1, 2, 2)
  plot_graphs(history, 'loss')
  plt.ylim(0, None)

In [ ]:
def test(model, train_dataset, validation_dataset, epochs):
  history = model.fit(train_dataset, epochs=epochs,
                      validation_data=validation_dataset,
                      #validation_steps=10
                      )

  validation_loss, validation_acc = model.evaluate(validation_dataset)

  print('Validation Loss:', validation_loss)
  print('Validation Accuracy:', validation_acc)

  sample_text = [
    'This is a proper sentence.',
    'These is an less proper sentences.',
    '.sentence bad a is This'
  ]
#  predictions = model.predict(np.array(sample_text, dtype=str))
#  for text, pred in zip([sample_text], predictions):
#    print(f"{text:10} >> {pred}")

  plt.figure(figsize=(16, 8))
  plt.subplot(1, 2, 1)
  plot_graphs(history, 'accuracy')
  plt.ylim(None, 1)
  plt.subplot(1, 2, 2)
  plot_graphs(history, 'loss')
  plt.ylim(0, None)

  test(model, train_dataset, validation_dataset, epochs=0)

## Data

In [ ]:
dataset, info = tfds.load('glue/cola', with_info=True)
original_train_dataset, original_validation_dataset, original_test_dataset = dataset['train'], dataset['validation'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/8551 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1043 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/glue/cola/incomplete.CY9DRD_2.0.0/glue-validation.tfrecord*...:   0%|     …

Generating test examples...:   0%|          | 0/1063 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/glue/cola/incomplete.CY9DRD_2.0.0/glue-test.tfrecord*...:   0%|          |…

Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/cola/2.0.0. Subsequent calls will reuse this data.


In [ ]:
print(original_train_dataset.element_spec)
for ex in original_train_dataset.take(1):
  print(type(ex))
  print(ex.keys())
  print('texts: ', ex['sentence'].numpy())
  print('labels: ', ex['label'].numpy())

{'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sentence': TensorSpec(shape=(), dtype=tf.string, name=None)}
<class 'dict'>
dict_keys(['idx', 'label', 'sentence'])
texts:  b'It is this hat that it is certain that he was wearing.'
labels:  1


In [ ]:
BUFFER_SIZE = 500
BATCH_SIZE = 64

train_dataset = original_train_dataset.map(lambda ex: (ex['sentence'], ex['label'])).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset = original_validation_dataset.map(lambda ex: (ex['sentence'], ex['label'])).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = original_test_dataset.map(lambda ex: (ex['sentence'], ex['label'])).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'Smith envied his good fortune to Jones.'
 b'The person who John asked for help thinks he is foolish.'
 b'I deny that that Bob has some money is certain.']

labels:  [0 1 0]


In [ ]:
for ex, lab in train_dataset.take(2):
  print(ex)
  print(lab)

tf.Tensor(
[b'George mailed the attorney his photograph of the accident.'
 b'The inspector analyzed the soundness in the building.'
 b'It is order one first that you should do first.'
 b'Cedric danced a jolly jig.' b'Which boy and the girl embraced?'
 b'John is more embarrassed, the more pictures of himself appear in the news.'
 b"Mary won't have been eating cake, but John."
 b'The car has been being driven.'
 b'The branches knocked against the wall.'
 b'Reports the covers of which the government prescribes the height of the lettering on almost always put me to sleep.'
 b"It's extremely windy today." b'The man chuckles'
 b'The perception of the problem is quite thorough.'
 b'Every man who said he would buy some salmon did buy some salmon.'
 b"You'll soon be owning all the land round here."
 b'We consider Leslie rather foolish, and Lou a complete idiot.'
 b'I will go and buy some whiskey.'
 b'The man I think chased Fido returned.'
 b'Shakespeare wrote his plays a long time ago.'
 b'They

### Question
> What's the impact of `map(lambda ex: (ex['sentence'], ex['label']))` in previous cell?

R1 : Récupère chaque liste de phrase et l'associe à une liste de label de meme taille

## Classification with a Bi-LSTM

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'to', 'a', 'that', 'i', 'is', 'john', 'of',
       'and', 'was', 'in', 'he', 'you', 'it', 'be', 'who', 'mary', 'on'],
      dtype='<U12')

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[208,   1,  28, 220,   1,   3, 480,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  2, 354,  17,   8, 142,  20, 440, 169,  13,   7,   1,   0,   0,
          0,   0],
       [  6,   1,   5,   5, 255,  37,  60, 149,   7, 244,   0,   0,   0,
          0,   0]])

### Question
> One may reconstruct a text from its encoded version (see the example below). What are the differences between the original text and the reconstructed one?

In [ ]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'Smith envied his good fortune to Jones.'
Round-trip:  smith [UNK] his good [UNK] to jones        

Original:  b'The person who John asked for help thinks he is foolish.'
Round-trip:  the person who john asked for help thinks he is [UNK]    

Original:  b'I deny that that Bob has some money is certain.'
Round-trip:  i [UNK] that that bob has some money is certain     



### Question

> What means the argument `from_logits=True` of the function `compile()` below?

R3 : Cela signifie que la sortie du modèle n'a pas d'activation sigmoid ou softmax et la perte l'applique automatiquement

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

tf.keras.utils.plot_model(model)

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
fit_and_test(model, train_dataset, validation_dataset, epochs=10)

134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - accuracy: 0.3097 - loss: 0.6731 - val_accuracy: 0.6251 - val_loss: 0.6178
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6083 - loss: 0.6161
Validation Loss: 0.6177969574928284
Validation Accuracy: 0.6251198649406433


ValueError: Invalid dtype: str3328

### Question

> When would you stop the iterations of the gradient descent above? Why?

In [ ]:
examples = [
    'This is a proper sentence.',
    'These is an less proper sentences.',
    '.sentence bad a is This'
]

predictions = tf.sigmoid(model(tf.constant(examples)))

for text, pred in zip(examples, predictions):
  print(f'input: {text:<30} : score: {pred[0]:.6f}')

## Classification with BERT

In [ ]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1'
bert_model = hub.KerasLayer(tfhub_handle_encoder, trainable=False)

bert_results = bert_model(bert_preprocess_model(text_test))

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
encoder_inputs = bert_preprocess_model(text_input)
outputs = bert_model(encoder_inputs)
net = tf.keras.layers.Dense(1)(outputs['pooled_output'])
model = tf.keras.Model(text_input, net)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

tf.keras.utils.plot_model(model)

In [ ]:
fit_and_test(model, train_dataset, validation_dataset, epochs=5)

### Question
> Comment these curves

In [ ]:
examples = [
    'This is a proper sentence.',
    'These is an less proper sentences.',
    '.sentence bad a is This'
]

predictions = tf.sigmoid(model(tf.constant(examples)))

for text, pred in zip(examples, predictions):
  print(f'input: {text:<30} : score: {pred[0]:.6f}')

## Short version

### Question

> add below a version of the codes limited to the code required to
> * load the data
> * build the model, train the model, and display the learning curves and the final validation loss and accuracy

In [ ]:
# load the data

In [ ]:
# build the model, train the model, and display the learning curves and the final validation loss and accuracy

## You own model

### Question
> Implement one of the following idea and comment its results:
> * Stack several BiLSTM. It requires the "hidden" Bi-LSTM to output one tensor per input token, whihc is obtained with the parameter `return_sequences=True`
> * Add a BiLSTM on-top of Bert. This BiLSTM has to use the output of the LSTM which corresponds to a tensor per input token (see `['sequence_output']`).
